# ABS Quarterly Wage Price Index 6345

Note: the WPI follows price changes in a fixed "basket" of jobs.

## Python set-up

In [1]:
# system imports
import re
import textwrap

# analytic imports
import pandas as pd
from readabs import metacol

# local imports
from abs_helper import get_abs_data
from plotting import (
    plot_growth_finalise,
    calc_growth,
    line_plot,
)

# pandas display settings
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("6345.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

In [3]:
textwrap.wrap(", ".join(abs_dict.keys()))

['634501, 634502a, 634502b, 634503a, 634503b, 634504a, 634504b, 634505a,',
 '634505b, 634507a, 634507b, 634508a, 634508b, 634509a, 634509b,',
 '63450Table2bto9b, 63450Table2ato9a']

## Plot

### Headline WPI for Australia

In [4]:
def headline() -> tuple[dict[str, pd.Series], str, str]:
    """Plot the headlines.
    Return a dictionary of the data plotted."""

    table = "634501"
    data = abs_dict[table]
    series_type = "Original"
    select = "Percentage Change from Corresponding Quarter of Previous Year"
    selected = meta[
        (meta[metacol.table] == table)
        & (meta[metacol.stype] == series_type)
        & meta[metacol.did].str.contains(select)
    ]

    text = "Total hourly rates of pay excluding bonuses ;  Australia"
    box = {}  # used for public v private comparative charts below

    for series_id, desc in zip(selected[metacol.id], selected[metacol.did]):
        # get the title
        title = (
            desc.replace(select, "")
            .replace(text, "")
            .replace(";", "")
            .replace("Private and Public", "All sectors")
            .strip()
        )
        title = re.sub(" +", " ", title)
        series = data[series_id].dropna()
        box[title] = series

        line_plot(
            series,
            starts=plot_times,
            tags=plot_tags,
            title=f"Annual Wage Price Growth: {title}",
            ylabel="Per cent per annum",
            rfooter=f"{source} {table}",
            lfooter=f'{text.replace(" ; ", ".")}. {series_type} series.',
            width=2,
            show=SHOW,
        )
    return box, text, series_type


headlines, name, stype = headline()

### WPI Public vs Private comparative charts

Around 77% of paid wages are in the private sector.

In [5]:
def pub_v_priv(box: dict[str, pd.Series], text: str, series_type: str) -> None:
    """Public v Private sector wage growth."""

    plot_data = pd.DataFrame(box)
    title = "Annual Wage Price Growth"
    line_plot(
        plot_data[plot_data.columns[:2]],
        starts=plot_times,
        tags=plot_tags,
        title=title,
        ylabel="Per cent per annum",
        rfooter=f"{source}",
        lfooter=f'{text.replace(" ; ", ".")}. {series_type} series.',
        width=2,
        show=SHOW,
    )


pub_v_priv(headlines, name, stype)

## WPI Growth Charts

In [6]:
def wpi_growth() -> None:
    """Plot WPI Growth."""

    table = "634501"
    data = abs_dict[table]

    for series_type in ("Original", "Seasonally Adjusted"):
        select = "Index"
        selected = meta[
            (meta[metacol.table] == table)
            & (meta[metacol.stype] == series_type)
            & meta[metacol.did].str.contains(select)
        ]
        for title, series_id in zip(selected[metacol.did], selected[metacol.id]):
            title = title.replace(
                "Quarterly Index ;  Total hourly rates "
                "of pay excluding bonuses ;  Australia ;  ",
                "",
            )
            title = title.replace(" ;", "")
            title = title.replace("  ", " ")
            type_txt = {"Original": "Orig", "Seasonally Adjusted": "Seas Adj"}[
                series_type
            ]
            title = f"WPI Growth: {title} ({type_txt})"
            series = data[series_id]
            growth = calc_growth(series)
            plot_growth_finalise(
                *growth,
                from_=pd.Period(plot_times[1], freq="Q"),
                title=title,
                rfooter=f"{source} {table}",
                lfooter="Australia. WPI = Wage Price Index. ",
                annotate=9,
                show=SHOW,
            )


wpi_growth()

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark --python --machine --iversion --conda --watermark

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

conda environment: n/a

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas: 2.2.3
re    : 2.2.1

Watermark: 2.5.0



In [8]:
print("Finished")

Finished
